## Description：
这个笔记本尝试实现一下项亮推荐系统实践里面的ItemCF算法， 采用的数据集是GroupLens提供的MovieLens的其中一个小数据集ml-latest-small。 该数据及包含700个用户对带有6100个标签的10000部电影的100000条评分。 该数据集是一个评分数据集， 用户可以给电影评5个不同等级的分数(1-5)， 而由于我们主要是研究隐反馈数据中的topN推荐问题， 所以忽略了数据集中的评分记录。  **TopN推荐的任务是预测用户会不会对某部电影评分， 而不是预测用户在准备对某部电影评分的前提下给电影评多少分**， 下面我们开始， 从逻辑上看， 其实这个任务主要分为下面的步骤：
1. 导入数据， 读取文件得到"用户-电影"的评分数据， 并且分为训练集和测试集
2. 计算电影之间的相似度
3. 针对目标用户u， 找到其最相似的k个用户， 产生N个推荐
4. 产生推荐之后， 通过准确率、召回率和覆盖率等进行评估。

In [1]:
import random
import numpy as np
import pandas as pd

import math
from operator import itemgetter

## 读入数据
读取文件得到"用户-电影"的评分数据， 并且分为训练集和测试集， 这里的思想是首先给出数据存在的路径， 然后通过pandas读取数据， 然后遍历该数据集， 把相应的数据存放到字典中， 这里之所以会用字典， 是因为用户对电影的评分会存在大量的稀疏。 所以我们依然需要建立一个"{用户：{电影: 评分}}"的这样一个字典， 后面基于这个字典去计算相似度。 如果感觉下面的代码理解有困难， 可以先参考我给出的博客链接补一下基础。

In [2]:
data_path = './ml-latest-small/'
data = pd.read_csv(data_path+'ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
data.pivot(index='userId', columns='movieId', values='rating')   # 这样会发现有大量的稀疏， 所以才会用字典进行存放

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# 声明两个字典， 分别是训练集和测试集
trainSet, testSet = {}, {}
trainSet_len, testSet_len = 0, 0
pivot = 0.75    # 训练集的比例

# 遍历data的每一行， 把userId, movidId, rating按照{user: {movidId: rating}}的方式存储， 当然定义一个随机种子进行数据集划分
for ele in data.itertuples():   # 遍历行这里推荐用itertuples， 比iterrows会高效很多
    user, movie, rating = getattr(ele, 'userId'), getattr(ele, 'movieId'), getattr(ele, 'rating')
    if random.random() < pivot:
        trainSet.setdefault(user, {})
        trainSet[user][movie] = rating
        trainSet_len += 1
    else:
        testSet.setdefault(user, {})
        testSet[user][movie] = rating 
        testSet_len += 1

print('Split trainingSet and testSet success!')
print('TrainSet = %s' % trainSet_len)
print('TestSet = %s' % testSet_len)

Split trainingSet and testSet success!
TrainSet = 75775
TestSet = 25061


## 计算电影之间的相似度
和UserItemCF相似， 这里同样需要建立一个倒排表， 只不过这里的倒排变成了{用户：物品}的倒排表， 如下：

![](./images/3.png)

而比较巧的是， 我们这里的存储正好是“用户-物品"评分表， 所以现在正好是倒排的形式， 所以不用刻意建立建立倒排表， 直接遍历trainSet即可， 但是在这之前， 我们先计算下每部电影的流行程度， 也就是被用户观看的总次数， 这个在衡量相似度的时候作为分母， 这里的其他逻辑和UserCF基本一致了。


In [5]:
# 计算每部电影的流行程度, 也就是每部电影被用户看的总次数， 这个衡量相似度的时候作为分母
movie_popular = {}
for user, movies in trainSet.items():   # 这里的user就是每个用户， movies还是个字典， {movieID: rating}
    for movie in movies:       # 这里的movie就是movieID了
        if movie not in movie_popular:     
            movie_popular[movie] = 0  
        movie_popular[movie] += 1


movie_count = len(movie_popular)
print('Total movie number = %d' % movie_count)


# 下面建立电影相似矩阵
print('Build user co-rated movies matrix ...')
movie_sim_matrix = {}
for user, movies in trainSet.items():     # 注意这个地方与UserCF的不同
    for m1 in movies:           # 对于每个电影， 都得双层遍历
        for m2 in movies:
            if m1 == m2:
                continue
            movie_sim_matrix.setdefault(m1, {})      # 把字典的值设置为字典的形式
            movie_sim_matrix[m1].setdefault(m2, 0)
            movie_sim_matrix[m1][m2] += 1     # 这里统计两个电影被同一个用户产生行为的次数， 这个就是余弦相似度的分子
print('Build user co-rated movies matrix success!')

# 下面计算电影之间的相似性
print('Calculating movies similarity matrix ...')
for m1, related_movies in movie_sim_matrix.items():
    for m2, count in related_movies.items():    # 这里面m2是相关电影， count是共同被同一个用户打分的次数
        # 这里注意零向量的处理， 即某电影的用户数为0
        if movie_popular[m1] == 0  or movie_popular[m2] == 0:
            movie_sim_matrix[m1][m2] = 0
        else:
            movie_sim_matrix[m1][m2] = count / math.sqrt(movie_popular[m1] * movie_popular[m2])  
print('Calculate movies similarity matrix success!')

Total movie number = 8776
Build user co-rated movies matrix ...
Build user co-rated movies matrix success!
Calculating movies similarity matrix ...
Calculate movies similarity matrix success!


## 针对目标用户u， 找到其最相似的k个用户， 产生N个推荐
得到物品相似度后， ItemCF算法通过下面公式计算用户u对物品j的兴趣：
$$p_{u j}=\sum_{i \in N(u) \cap S(j, K)} w_{j i} r_{u i}$$
其中， $S(j,k)$是和物品j最相似的K个物品的集合， $N(u)$是用户喜欢的物品的集合， $w_{ji}$是物品j和物品i的相似度， $r_{ui}$代表用户u对物品i的兴趣， 因为使用单一行为的隐反馈数据， 所以这里$r_{ui}=1$, 该公式的含义是， 和用户历史上感兴趣的物品越相似的物品， 越有可能在用户的推荐列表中获得较高的排名<br><br>

所以下面的代码逻辑是这样：
* 首先， 给定我一个用户ID， 我先拿到这个用户ID目前看过的所有电影， 以防后面推荐重了。  
* 然后从相似性矩阵中，找到与当前用户看的物品的最相近的K个物品
* 遍历他们看过的电影， 如果当前用户没有看过， 该电影的权重等级累加
* 最后给所有的电影进行排序， 推荐前n部给当前用户

In [9]:
# 找到最相似的K个产品， 并推荐给n个用户
k = 20
n = 10

aim_user = 10     # 目标用户ID
rank ={}
watched_movies = trainSet[aim_user]      # 找出目标用户看到电影

for movie, rating in watched_movies.items():
    #遍历与物品item最相似的前k个产品，获得这些物品及相似分数
    for related_movie, w in sorted(movie_sim_matrix[movie].items(), key=itemgetter(1), reverse=True)[:k]:
        # 若该物品用户看过， 跳过
        if related_movie in watched_movies:
            continue
        
        # 计算用户user对related_movie的偏好值， 初始化该值为0
        rank.setdefault(related_movie, 0)
        #通过与其相似物品对物品related_movie的偏好值相乘并相加。
        #排名的依据—— > 推荐电影与该已看电影的相似度(累计) * 用户对已看电影的评分
        rank[related_movie] += w * float(rating)
    
# 产生最后的推荐列表
rec_movies = sorted(rank.items(), key=itemgetter(1), reverse=True)[:n]  # itemgetter(1) 是简洁写法

In [10]:
rec_movies

[(6539, 17.105560519267044),
 (2571, 15.319812012008285),
 (5989, 15.167246866323042),
 (8961, 14.851552717287301),
 (4993, 13.460889880196493),
 (45722, 13.42772434176251),
 (5349, 13.007947819168727),
 (5418, 11.127470241881547),
 (58559, 11.045109815505814),
 (59258, 10.429545216130437)]

## 产生推荐之后， 通过准确率、召回率和覆盖率等进行评估。
这里介绍评测指标：<br><br>
1. 召回率<br>
对用户u推荐N个物品记为$R(u)$, 令用户u在测试集上喜欢的物品集合为$T(u)$， 那么召回率定义为：
$$\operatorname{Recall}=\frac{\sum_{u}|R(u) \cap T(u)|}{\sum_{u}|T(u)|}$$
这个意思就是说， 在用户真实购买或者看过的影片里面， 我模型真正预测出了多少， 这个考察的是模型推荐的一个全面性。 <br>

2. 准确率<br>
准确率定义为：
$$\operatorname{Precision}=\frac{\sum_{u} \mid R(u) \cap T(u)}{\sum_{u}|R(u)|}$$
这个意思再说， 在我推荐的所有物品中， 用户真正看的有多少， 这个考察的是我模型推荐的一个准确性。 <br><br>
为了提高准确率， 模型需要把非常有把握的才对用户进行推荐， 所以这时候就减少了推荐的数量， 而这往往就损失了全面性， 真正预测出来的会非常少，所以实际应用中应该综合考虑两者的平衡。

3. 覆盖率
覆盖率反映了推荐算法发掘长尾的能力， 覆盖率越高， 说明推荐算法越能将长尾中的物品推荐给用户。
$$\text { Coverage }=\frac{\left|\bigcup_{u \in U} R(u)\right|}{|I|}$$
该覆盖率表示最终的推荐列表中包含多大比例的物品。如果所有物品都被给推荐给至少一个用户， 那么覆盖率是100%。

4. 新颖度
用推荐列表中物品的平均流行度度量推荐结果的新颖度。 如果推荐出的物品都很热门， 说明推荐的新颖度较低。  由于物品的流行度分布呈长尾分布， 所以为了流行度的平均值更加稳定， 在计算平均流行度时对每个物品的流行度取对数。

In [14]:
# 这里先把产生推荐的那个封装成函数才能测试所有的测试样本
def recommend(aim_user, k=20, n=10):
    rank ={}
    watched_movies = trainSet[aim_user]      # 找出目标用户看到电影

    for movie, rating in watched_movies.items():
        #遍历与物品item最相似的前k个产品，获得这些物品及相似分数
        for related_movie, w in sorted(movie_sim_matrix[movie].items(), key=itemgetter(1), reverse=True)[:k]:
            # 若该物品用户看过， 跳过
            if related_movie in watched_movies:
                continue

            # 计算用户user对related_movie的偏好值， 初始化该值为0
            rank.setdefault(related_movie, 0)
            #通过与其相似物品对物品related_movie的偏好值相乘并相加。
            #排名的依据—— > 推荐电影与该已看电影的相似度(累计) * 用户对已看电影的评分
            rank[related_movie] += w * float(rating)

    # 产生最后的推荐列表
    return sorted(rank.items(), key=itemgetter(1), reverse=True)[:n]  # itemgetter(1) 是简洁写法

In [15]:
# 准确率、召回率和覆盖率
hit = 0
rec_count = 0     # 统计推荐的影片数量， 计算查准率
test_count = 0    # 统计测试集的影片数量， 计算查全率
all_rec_movies = set()    # 统计被推荐出来的影片个数， 无重复了， 为了计算覆盖率
item_populatity = dict()   # 计算新颖度

# 先计算每部影片的流行程度
for user, items in trainSet.items():
    for item in items.keys():
        if item not in item_populatity:
            item_populatity[item] = 0
        item_populatity[item] += 1    # 这里统计训练集中每部影片用户观看的总次数， 代表每部影片的流行程度


# 计算评测指标
ret = 0
ret_cou = 0
for user, items in trainSet.items():    # 这里得保证测试集里面的用户在训练集里面才能推荐
    
    test_movies = testSet.get(user, {})
    rec_movies = recommend(user)
    for movie, w in rec_movies:
        if movie in test_movies:
            hit += 1
        all_rec_movies.add(movie)
        ret += math.log(1+item_populatity[movie])
        ret_cou += 1
    rec_count += n
    test_count += len(test_movies)
    
    
precision = hit / (1.0 * rec_count)
recall = hit / (1.0 * test_count)
coverage = len(all_rec_movies) / movie_count
ret /= ret_cou*1.0
    
print('precisioin = %.4f\nrecall = %.4f\ncoverage = %.4f\npopularity = %.4f' % (precision, recall, coverage, ret))

precisioin = 0.2738
recall = 0.0666
coverage = 0.0680
popularity = 4.5310
